In [ ]:
import pandas as pd
import networkx as nx
from pathlib import Path
import yaml

In [ ]:
df_manual = pd.read_excel("assets/data/Table_S1.xlsx", index_col=0, converters={'version':str})

In [ ]:
G = nx.read_graphml("assets/data/Figure_2/rulegrah_numbers.graphml")
G2 = nx.read_graphml("assets/data/Figure_2/rulegraph_tidy_fin.graphml")

In [ ]:
sup_table_1 = {}

for n in [n for n in G2.nodes]:
    x1,y1 = G.nodes[n]['x'], G.nodes[n]['y']
    x2,y2 = G2.nodes[n]['x'], G2.nodes[n]['y']
    assert x1 == x2
    assert y1 == y2
    label = G2.nodes[n]['label']
    number = G.nodes[n]['label']
    sup_table_1[number] = {'Rule Name' : label}

sup_table_1

In [ ]:
df = pd.DataFrame.from_dict(sup_table_1).T
df2 = df.reindex(columns = df.columns.tolist() + ["Tool Name", "Version", "Description", "Link", "References"])
df2.T.to_json("assets/data/Table_S1.json")
df2['Rule Name']

In [ ]:
paper = [3, 6, 11, 5, 4, 14, 31, 22, 36, 55, 48, 27, 34, 44]
keep_rule = []
for i in df_manual.index:
    rule_name = df_manual.loc[i, "Rule Name"]
    
    target = df2[df2['Rule Name'] == rule_name].index
    for c in df_manual.columns:
        df2.loc[target, c] = df_manual.loc[i, c]
    df_manual.loc[i, "node_number"] = target
    print(i, target, rule_name, rule_name in list(df2['Rule Name']))
    if target in [str(i) for i in paper]:
        pass
        keep_rule.append(i)
    else:
        print(f"drop {rule_name}")
df_table_s1 = df_manual.loc[keep_rule].copy().reset_index(drop=True)
df_table_s1

In [ ]:
df_table_s1.to_excel("assets/supplementary_materials/TABLE_S1.xlsx", index=None)

In [ ]:
df2.index.name = "node_number"
df2.to_excel("assets/supplementary_materials/TABLE_S2.xlsx", index=None)

In [ ]:
rule_dir = Path("../workflow/rules/")
smk = rule_dir / "bgc_analytics.smk"